In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from collections import Counter
import re
from nltk.corpus import stopwords
import nltk
import plotly.express as px

In [7]:
calendar_df = pd.read_csv('archive/calendar.csv')
listings_df = pd.read_csv('archive/listings.csv')
reviews_df = pd.read_csv('archive/reviews.csv')

In [8]:
pd.set_option('display.max_columns', None)

In [9]:
calendar_df.head()

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN
2,12147973,2017-09-03,f,NaN
3,12147973,2017-09-02,f,NaN
4,12147973,2017-09-01,f,NaN


In [10]:
reviews_df.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1178162,4724140,2013-05-21,4298113,Olivier,My stay at islam's place was really cool! Good...
1,1178162,4869189,2013-05-29,6452964,Charlotte,Great location for both airport and city - gre...
2,1178162,5003196,2013-06-06,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...
3,1178162,5150351,2013-06-15,2215611,Marine,The room was nice and clean and so were the co...
4,1178162,5171140,2013-06-16,6848427,Andrew,Great location. Just 5 mins walk from the Airp...


In [11]:
listings_df.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,access,interaction,house_rules,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,20160906204935,2016-09-07,Sunny Bungalow in the City,"Cozy, sunny, family home. Master bedroom high...",The house has an open and cozy feel at the sam...,"Cozy, sunny, family home. Master bedroom high...",none,"Roslindale is quiet, convenient and friendly. ...",NaN,"The bus stop is 2 blocks away, and frequent. B...","You will have access to 2 bedrooms, a living r...",NaN,Clean up and treat the home the way you'd like...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,https://a2.muscache.com/im/pictures/c0842db1-e...,31303940,https://www.airbnb.com/users/show/31303940,Virginia,2015-04-15,"Boston, Massachusetts, United States",We are country and city connecting in our deck...,NaN,NaN,NaN,f,https://a2.muscache.com/im/pictures/5936fef0-b...,https://a2.muscache.com/im/pictures/5936fef0-b...,Roslindale,1,1,"['email', 'phone', 'facebook', 'reviews']",t,f,"Birch Street, Boston, MA 02131, United States",Roslindale,Roslindale,NaN,Boston,MA,02131,Boston,"Boston, MA",US,United States,42.282619,-71.133068,t,House,Entire home/apt,4,1.5,2.0,3.0,Real Bed,"{TV,""Wireless Internet"",Kitchen,""Free Parking ...",NaN,$250.00,NaN,NaN,NaN,$35.00,1,$0.00,2,1125,2 weeks ago,NaN,0,0,0,0,2016-09-06,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,20160906204935,2016-09-07,Charming room in pet friendly apt,Charming and quiet room in a second floor 1910...,Small but cozy and quite room with a full size...,Charming and quiet room in a second floor 1910...,none,"The room is in Roslindale, a diverse and prima...","If you don't have a US cell phone, you can tex...",Plenty of safe street parking. Bus stops a few...,Apt has one more bedroom (which I use) and lar...,"If I am at home, I am likely working in my hom...",Pet friendly but please confirm with me if the...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,https://a1.muscache.com/im/pictures/39327812/d...,2572247,https://www.airbnb.com/users/show/2572247,Andrea,2012-06-07,"Boston, Massachusetts, United States",I live in Boston and I like to travel and have...,within an hour,100%,100%,f,https://a2.muscache.com/im/users/2572247/profi...,https://a2.muscache.com/im/users/2572247/profi...,Roslindale,1,1,"['email', 'phone', 'facebook', 'linkedin', 'am...",t,t,"Pinehurst Street, Boston, MA 02131, United Sta

In [12]:
list(pd.unique(calendar_df['price']))

[nan,
 '$65.00',
 '$75.00',
 '$79.00',
 '$100.00',
 '$125.00',
 '$118.00',
 '$200.00',
 '$113.00',
 '$115.00',
 '$114.00',
 '$122.00',
 '$135.00',
 '$58.00',
 '$69.00',
 '$309.00',
 '$229.00',
 '$269.00',
 '$60.00',
 '$40.00',
 '$44.00',
 '$41.00',
 '$57.00',
 '$150.00',
 '$145.00',
 '$165.00',
 '$170.00',
 '$49.00',
 '$45.00',
 '$120.00',
 '$70.00',
 '$95.00',
 '$90.00',
 '$110.00',
 '$67.00',
 '$68.00',
 '$55.00',
 '$250.00',
 '$119.00',
 '$177.00',
 '$176.00',
 '$300.00',
 '$166.00',
 '$172.00',
 '$167.00',
 '$168.00',
 '$169.00',
 '$185.00',
 '$179.00',
 '$178.00',
 '$163.00',
 '$184.00',
 '$181.00',
 '$183.00',
 '$182.00',
 '$190.00',
 '$191.00',
 '$189.00',
 '$197.00',
 '$195.00',
 '$194.00',
 '$188.00',
 '$186.00',
 '$193.00',
 '$196.00',
 '$180.00',
 '$192.00',
 '$175.00',
 '$174.00',
 '$159.00',
 '$225.00',
 '$400.00',
 '$220.00',
 '$219.00',
 '$206.00',
 '$203.00',
 '$205.00',
 '$202.00',
 '$204.00',
 '$201.00',
 '$207.00',
 '$210.00',
 '$213.00',
 '$212.00',
 '$209.00',
 '$2

In [13]:
calendar_df['price'] = pd.to_numeric(calendar_df['price'].str.replace('$','').str.replace(',',''))

In [14]:
listings_df['price'] = pd.to_numeric(listings_df['price'].str.replace('$','').str.replace(',',''))

In [15]:
calendar_df.isna().sum()

listing_id         0
date               0
available          0
price         665853
dtype: int64

In [16]:
calendar_df[pd.isna(calendar_df['price'])]

,listing_id,date,available,price
0,12147973,2017-09-05,f,NaN
1,12147973,2017-09-04,f,NaN
2,12147973,2017-09-03,f,NaN
3,12147973,2017-09-02,f,NaN
4,12147973,2017-09-01,f,NaN
...,...,...,...,...
1308885,14504422,2016-09-10,f,NaN
1308886,14504422,2016-09-09,f,NaN
1308887,14504422,2016-09-08,f,NaN
1308888,14504422,2016-09-07,f,NaN


In [17]:
calendar_df['date'] = pd.to_datetime(calendar_df['date'])

In [18]:
calendar_df[calendar_df['available']=='f'].groupby(calendar_df.date.dt.strftime('%B'))['available'].value_counts().sort_values(ascending=False)

date       available
September  f            74182
October    f            64821
May        f            57566
July       f            56203
April      f            56198
March      f            55985
August     f            55605
June       f            54849
December   f            50169
November   f            48692
January    f            47985
February   f            43598
Name: count, dtype: int64

In [19]:
print('Price Mean: {}'.format(calendar_df['price'].mean()))
print('Price Median: {}'.format(calendar_df['price'].median()))
print('Price Stdev: {}'.format(calendar_df['price'].std()))
print('Q1 Percentile: {}'.format(np.percentile(calendar_df[calendar_df['available']=='t']['price'],25)))
print('Q3 Percentile: {}'.format(np.percentile(calendar_df[calendar_df['available']=='t']['price'],75)))

Price Mean: 198.43890942511862
Price Median: 150.0
Price Stdev: 180.1429126916303
Q1 Percentile: 85.0
Q3 Percentile: 255.0


In [20]:
# Ensure that NLTK's stopwords are available; if not, download them
nltk.download('stopwords')

# Sample DataFrame creation
data = {
    'listing_id': [1178162, 1178162, 1178162],
    'id': [4724140, 4869189, 5003196],
    'date': ['2013-05-21', '2013-05-29', '2013-06-06'],
    'reviewer_id': [4298113, 6452964, 6449554],
    'reviewer_name': ['Olivier', 'Charlotte', 'Sebastian'],
    'comments': [
        "My stay at islam's place was really cool! Good...",
        "Great location for both airport and city - gre...",
        "We really enjoyed our stay at Islams house. Fr."
    ]
}


# Combine all comments into a single string
all_comments = ' '.join(reviews_df[pd.notna(reviews_df['comments'])]['comments'])

# Preprocess comments: lowercasing, removing punctuation and splitting into words
words = re.findall(r'\b\w+\b', all_comments.lower())

# Filter out stopwords
filtered_words = [word for word in words if word not in stopwords.words('english')]

# Count the frequency of each word
word_counts = Counter(filtered_words)

# Get the most common words
most_common_words = word_counts.most_common()

print(most_common_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aalgu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[('great', 40395), ('stay', 35495), ('boston', 32715), ('place', 30398), ('apartment', 28450), ('clean', 23199), ('location', 23035), ('us', 21575), ('room', 20181), ('nice', 19026), ('would', 18924), ('host', 17931), ('comfortable', 15976), ('house', 14532), ('everything', 13886), ('easy', 13117), ('really', 12612), ('recommend', 12471), ('time', 12338), ('well', 11747), ('good', 11575), ('home', 11406), ('perfect', 10823), ('neighborhood', 10352), ('also', 10291), ('close', 10042), ('walk', 10017), ('definitely', 9979), ('made', 8874), ('get', 8750), ('helpful', 8680), ('experience', 8675), ('area', 7961), ('bed', 7612), ('one', 7373), ('even', 7340), ('staying', 7242), ('restaurants', 7216), ('back', 7073), ('needed', 6956), ('like', 6878), ('friendly', 6771), ('wonderful', 6766), ('quiet', 6749), ('beautiful', 6400), ('check', 6273), ('city', 6194), ('around', 6194), ('convenient', 6054), ('walking', 6007), ('could', 6000), ('night', 5930), ('station', 5886), ('kitchen', 5747), ('h

In [21]:
pd.Series(most_common_words).head(6)

0        (great, 40395)
1         (stay, 35495)
2       (boston, 32715)
3        (place, 30398)
4    (apartment, 28450)
5        (clean, 23199)
dtype: object

In [22]:
geo_df  = listings_df[['id','name','host_name','latitude','longitude','property_type','price']]

In [23]:
fig = px.scatter_mapbox(geo_df, 
                        lon=geo_df['longitude'],
                        lat = geo_df['latitude'],
                        zoom=3,
                        color = geo_df['property_type'],
                        size = geo_df['price'],
                        width=1200,
                        height=900,
                        title='Boston Listings Airbnbs')

fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(margin={'r':0,'t':50,'l':0,'b':10})

fig.show()